In [ ]:
###关于数据的保存：在bash运行——./save.sh "这里写修改说明"###

#===========================================================#
#####################总体思路梳理############################
#===========================================================#
##################模块一：得到目标论文序列####################
#STEP1————匹配
#将论文引用关系原始数据与撤稿论文序列进行匹配，目的是得到所有引用了撤稿论文的论文




# 前期准备模块

## 库的导入

In [1]:
###########################这一单元模块专门用于导入需要用到的库####################################

import pandas as pd #基础的数据处理
import os 
#Python 中的一条指令，用于导入内置的os模块。
#os模块提供了与操作系统交互的功能，允许你执行文件和目录的操作
#如路径管理、文件创建、删除等
import json #用于处理json数据的库
from tqdm import tqdm
import math # 用于检测 NaN
from collections import Counter #Python标准库collections里的的工具，用来统计元素出现的次数
from collections import defaultdict 
'''
collections 是 Python 自带的一个模块，
里面提供了许多有用的容器数据类型，比如 Counter、deque、namedtuple、defaultdict 等。
defaultdict 是一个字典的子类，它和普通字典类似，
但当你访问一个不存在的键时，它会自动帮你创建一个默认值，而不是抛出 KeyError。
'''
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import ast

## 查看大体量数据的前十行（CSV/EXCEL/JSON）

In [ ]:
###########数据前10行展示##############
######用来判断太大数据的数据结构########
#CSV文件：
file_csv = '/Data1/DATA/Dimensions2024/20240101/Papers.csv'
df_csv = pd.read_csv(file_csv, nrows=10)
df_csv

In [ ]:
#EXCEL文件：
file_excel = ''
df_excel = pd.read_excel(file_excel, nrows=10)
df_excel

In [ ]:
#JSON文件：
file_json = 'JSON文件路径.json'
df_json = pd.read_json(file_json)
df_json

## 查看CSV文件列名

In [ ]:
#看df文件的列名
print(NewE_C_10_10 .columns)

# 模块一：得到目标论文序列

## STEP1——匹配所有引用了撤稿论文的文献序列df_containretra

In [ ]:
#这个部分需要两个原始数据
#（1）原始数据1：论文与其参考文献的对应关系（/Data1/DATA/Dimensions2024/20240101/PaperReferences.csv）
'''
df数据结构：
    id	            reference_ids
0	pub.1038660625	pub.1043519304
1	pub.1038660625	pub.1012015875
2	pub.1038660625	pub.1031233780
3	pub.1038660625	pub.1027495706
'''
#（2）原始数据2：4w篇撤稿论文的原始数据（/Data4/yutao_wen/matched_output2.csv）
'''
df数据结构（列名与数据类型）
Index(['UT', 'Paper_doi', 'Note_doi', 'Title', 'Journal', 'ISOAbbr',
       'Publisher', 'Article', 'Review', 'Institute', 'City', 'Country',
       'First', 'RP', 'Micro_topics', 'Meso_topics', 'Macro_topics', '3Years',
       '5Years', 'Total', 'Fund', 'Year', 'RYear', 'publish_date',
       'retraction_date', 'Open Access', 'Tags', 'Type', 'note_URL', 'Authors',
       'Addresses', 'Affiliations', 'PaperID', 'id', 'title', 'doi', 'date',
       'journal.id'],
      dtype='object')
'''


### 原始数据1与2的导入——30MIN

In [2]:
#先将两个原始文件都导入内核中
original_file1 = '/Data1/DATA/Dimensions2024/20240101/PaperReferences.csv'
df_references = pd.read_csv(original_file1)
original_file2 = '/Data4/yutao_wen/matched_output2.csv'
df_retrapapers = pd.read_csv(original_file2)
#time约30min

### 使用布尔索引匹配2个df——12MIN

In [3]:
# 保证reference_ids和撤稿论文id列的数据类型一致（通常为字符串）
df_references['reference_ids'] = df_references['reference_ids'].astype(str)
df_retrapapers['id'] = df_retrapapers['id'].astype(str)

# 使用布尔索引保留reference_ids出现在撤稿论文id中的行
df_containretra = df_references[df_references['reference_ids'].isin(df_retrapapers['id'])].copy()
#time约12min

### 匹配产出文件df_containretra预览

In [5]:
df_containretra

,id,reference_ids
0,pub.1148484199,pub.1013663776
1,pub.1128687208,pub.1003140261
2,pub.1148009542,pub.1146057724
3,pub.1114696738,pub.1026667060
4,pub.1114902564,pub.1028915017
...,...,...
765834,pub.1134201694,pub.1113603600
765835,pub.1016972058,pub.1008362454
765836,pub.1085464507,pub.1039593849
765837,pub.1124870274,pub.1124369222


### 匹配文件保存备份

In [6]:
output_path = '/Data4/yutao_wen/25.9.17/df_containretra.csv'
# 保存为 CSV 文件
df_containretra.to_csv(output_path, index=False)

### 导入df_containretra文件

In [29]:
df_containretra = pd.read_csv("/Data4/yutao_wen/25.9.17/df_containretra.csv")

## STEP2——匹配撤稿论文/目标论文原始信息

In [ ]:
#撤稿论文主要是两个关键年份（发表/撤稿）以及期刊id较为重要————来源于matched_output2
#（1）RPYear：被引用的撤稿论文的发表年份
#（2）RYear：被引用的撤稿论文的撤稿年份
#（3）RJournal_id：被引用的撤稿论文的期刊ID
#目标论文主要是发表年份与期刊id较为重要————来源于Papers.csv
#（1）Year：目标论文的发表年份
#（2）Journal_id：目标论文的期刊ID

### 原始数据2导入

In [30]:
original_file2 = '/Data4/yutao_wen/matched_output2.csv'
df_retrapapers = pd.read_csv(original_file2)

### 匹配撤稿论文RPYear、RYear、RJournal_id

In [31]:
#从 df_retrapapers 中选择需要匹配的列，并改列名
df_retra_subset = df_retrapapers[['id', 'Year', 'RYear', 'journal.id']].copy()
df_retra_subset = df_retra_subset.rename(columns={
    'id': 'reference_ids',
    'Year': 'RPYear',
    'RYear': 'RYear',
    'journal.id': 'RJournal_id'
})

# 第三步：将信息 merge 到 df_containretra 中
df_containretra = df_containretra.merge(df_retra_subset, on='reference_ids', how='left')

### 原始数据3导入——10min

In [11]:
# 此时需要第三个原始文件：Papers.csv文件
# 其蕴含了数据库所有论文的各种信息
original_file3 = '/Data1/DATA/Dimensions2024/20240101/Papers.csv'
'''
数据结构：
Index(['id', 'title', 'doi', 'date', 'date_inserted', 'date_normal',
       'date_online', 'date_print', 'type', 'journal.id', 'journal.title',
       'times_cited', 'num_funds', 'num_authors', 'num_references', 'field_l0',
       'field_l1', 'resulting_publication_doi'],
      dtype='object')

'''
#获取目标论文的Year与Journal.id
df_papers = pd.read_csv(original_file3,usecols=['id', 'date_normal','journal.id'])
#time约10min

### 详细年月日date提取年份Year

In [12]:
# 提取年份（ date 格式如 '2019-03-15'）
df_papers['Year'] = pd.to_datetime(df_papers['date_normal'], errors='coerce').dt.year

### 匹配目标论文Year、Journal.id——4MIN

In [32]:
# 只选择需要的列，重命名 'journal.id' 为 'Journal_id'
df_papers_subset = df_papers[['id', 'Year','journal.id']].rename(columns={'journal.id': 'Journal_id'})

# 合并，添加 Journal_id 列
df_containretra = df_containretra.merge(df_papers_subset, on='id', how='left')

In [36]:
# 调整列顺序
df_containretra = df_containretra[['id', 'reference_ids', 'Year', 'Journal_id', 'RPYear', 'RYear', 'RJournal_id']]


In [37]:
df_containretra

,id,reference_ids,Year,Journal_id,RPYear,RYear,RJournal_id
0,pub.1148484199,pub.1013663776,2019.0,NaN,2014.0,2022.0,jour.1044393
1,pub.1128687208,pub.1003140261,2012.0,NaN,2000.0,2012.0,jour.1412991
2,pub.1148009542,pub.1146057724,2022.0,jour.1115238,2022.0,2023.0,jour.1043255
3,pub.1114696738,pub.1026667060,2015.0,jour.1050605,2015.0,2016.0,jour.1084523
4,pub.1114902564,pub.1028915017,2012.0,jour.1038288,1992.0,2016.0,jour.1077219
...,...,...,...,...,...,...,...
766248,pub.1134201694,pub.1113603600,2020.0,jour.1041030,2019.0,2023.0,jour.1036281
766249,pub.1016972058,pub.1008362454,2016.0,jour.1046517,2013.0,2018.0,jour.1327387
766250,pub.1085464507,pub.1039593849,2017.0,jour.1454776,2011.0,2018.0,jour.1082971
766251,pub.1124870274,pub.1124369222,2020.0,jour.1319730,2020.0,2020.0,jour.1095007


### 检查每列NaN情况

In [38]:
# 计算每列的 NaN 数量
nan_counts = df_containretra.isna().sum()

# 输出每列 NaN 的数量
print("每列 NaN 的数量：")
print(nan_counts)


每列 NaN 的数量：
id                   0
reference_ids        0
Year                84
Journal_id       71649
RPYear            3985
RYear             4786
RJournal_id      25646
dtype: int64


### 保留不含NaN的行

In [39]:
# 计算删除前 DataFrame 的总行数
total_rows_before = len(df_containretra)

# 删除含有 NaN 的行
df_containretra_cleaned = df_containretra.dropna(axis=0, how='any')

# 计算删除后每列的 NaN 数量
nan_counts_after = df_containretra_cleaned.isna().sum()

# 计算删除后的 DataFrame 的总行数
total_rows_after = len(df_containretra_cleaned)

# 计算删除的行数
rows_deleted = total_rows_before - total_rows_after

# 输出结果
print("\n删除了多少行（含有 NaN 的行）：", rows_deleted)
print("\n删除后每列 NaN 的数量：")
print(nan_counts_after)
print("\n删除前的行数：", total_rows_before)
print("删除后的行数：", total_rows_after)


删除了多少行（含有 NaN 的行）： 96608

删除后每列 NaN 的数量：
id               0
reference_ids    0
Year             0
Journal_id       0
RPYear           0
RYear            0
RJournal_id      0
dtype: int64

删除前的行数： 766253
删除后的行数： 669645


In [40]:
df_containretra_cleaned

,id,reference_ids,Year,Journal_id,RPYear,RYear,RJournal_id
2,pub.1148009542,pub.1146057724,2022.0,jour.1115238,2022.0,2023.0,jour.1043255
3,pub.1114696738,pub.1026667060,2015.0,jour.1050605,2015.0,2016.0,jour.1084523
4,pub.1114902564,pub.1028915017,2012.0,jour.1038288,1992.0,2016.0,jour.1077219
7,pub.1107325835,pub.1003143127,2018.0,jour.1030157,2006.0,2018.0,jour.1030157
8,pub.1011260253,pub.1032808220,2008.0,jour.1013016,2005.0,2007.0,jour.1013016
...,...,...,...,...,...,...,...
766247,pub.1130857724,pub.1117049318,2020.0,jour.1136301,2019.0,2023.0,jour.1138382
766248,pub.1134201694,pub.1113603600,2020.0,jour.1041030,2019.0,2023.0,jour.1036281
766249,pub.1016972058,pub.1008362454,2016.0,jour.1046517,2013.0,2018.0,jour.1327387
766250,pub.1085464507,pub.1039593849,2017.0,jour.1454776,2011.0,2018.0,jour.1082971


### 匹配文件保存与备份

In [41]:
output_path = '/Data4/yutao_wen/25.9.17/df_containretra2.csv'
# 保存为 CSV 文件
df_containretra_cleaned.to_csv(output_path, index=False)
'''
数据结构为：
	            id	reference_ids	Year	 Journal_id	    RPYear	RYear	RJournal_id
2	pub.1148009542	pub.1146057724	2022.0	jour.1115238	2022.0	2023.0	jour.1043255
3	pub.1114696738	pub.1026667060	2015.0	jour.1050605	2015.0	2016.0	jour.1084523
4	pub.1114902564	pub.1028915017	2012.0	jour.1038288	1992.0	2016.0	jour.1077219
7	pub.1107325835	pub.1003143127	2018.0	jour.1030157	2006.0	2018.0	jour.1030157
8	pub.1011260253	pub.1032808220	2008.0	jour.1013016	2005.0	2007.0	jour.1013016
...	...	...	...	...	...	...	...
'''

'\n数据结构为：\n\t            id\treference_ids\tYear\t Journal_id\t    RPYear\tRYear\tRJournal_id\n2\tpub.1148009542\tpub.1146057724\t2022.0\tjour.1115238\t2022.0\t2023.0\tjour.1043255\n3\tpub.1114696738\tpub.1026667060\t2015.0\tjour.1050605\t2015.0\t2016.0\tjour.1084523\n4\tpub.1114902564\tpub.1028915017\t2012.0\tjour.1038288\t1992.0\t2016.0\tjour.1077219\n7\tpub.1107325835\tpub.1003143127\t2018.0\tjour.1030157\t2006.0\t2018.0\tjour.1030157\n8\tpub.1011260253\tpub.1032808220\t2008.0\tjour.1013016\t2005.0\t2007.0\tjour.1013016\n...\t...\t...\t...\t...\t...\t...\t...\n'

## STEP3——筛选出合理目标论文

In [42]:
#主要是对STEP2的数据进行无关对象清除
#因为我们关注的目标论文是：
# （1）参考文献仅仅有一篇被撤稿
# （2）而且撤稿时间在2023年之前
# （3）引用论文的时候该被引论文尚未被撤稿
# （4）自己本身没被撤稿
#Exclude data ————— 主要有四部分被去掉
#(1) cited more than one retracted paper
#(2) retracted in 2023 or 2024
#(3) citing papers that were themselves retracted
#(4) The paper itself was retracted

### 导入STEP2终文件df_containretra2

In [43]:
df_containretra2 = '/Data4/yutao_wen/25.9.17/df_containretra2.csv'
df_containretra2 = pd.read_csv(df_containretra2)

### （1）删除参考文献有一篇以上被撤稿的

In [44]:
#(1)cited more than one retracted paper
#需要先去掉参考文献被撤稿了2篇及以上的目标论文，因为这部分的撤稿冲击存在多个时间点，分析复杂，暂不考虑
df_containretra_copy = df_containretra2.copy()

# 统计原始行数
original_count = len(df_containretra_copy)

# 删除所有出现两次及以上的 id 的行
df_containretra_copy = df_containretra_copy[~df_containretra_copy['id'].duplicated(keep=False)]

# 统计删减后行数
filtered_count = len(df_containretra_copy)

# 输出对比信息
print(f"原来有 {original_count} 行")
print(f"去除重复 id 后有 {filtered_count} 行")
print(f"共删除了 {original_count - filtered_count} 行")

原来有 669645 行
去除重复 id 后有 499051 行
共删除了 170594 行


### （2）删除参考文献在2023、2024被撤稿的

In [45]:
#(2)retracted in 2023 or 2024
#去掉2023年、2024年被撤稿的参考文献部分
#因为需要研究撤稿后的趋势，这两年撤稿的文章后续时期的数据太少了，不便于分析
# 创建副本，避免修改原始数据

# 记录原始行数
original_count = len(df_containretra_copy)

# 去掉 RPYear 为 2023 或 2024 的行
df_containretra_copy = df_containretra_copy[~df_containretra_copy['RYear'].isin([2023.0, 2024.0])]

# 记录删减后行数
filtered_count = len(df_containretra_copy)

# 输出行数信息
print(f"原来有 {original_count} 行")
print(f"删减后有 {filtered_count} 行")
print(f"共删除了 {original_count - filtered_count} 行")

原来有 499051 行
删减后有 363465 行
共删除了 135586 行


### （3）删除引用参考文献时就已被撤稿的

In [46]:
#(3)citing papers that were themselves retracted
#发现存在有的论文引用了已经被撤稿的论文，数学表现形式即———— RYear-Year < 0，需要删除
# 创建新列 RYear-Year
df_containretra_copy['RYear-Year'] = df_containretra_copy['RYear'] - df_containretra_copy['Year']

# 记录原始行数
original_count = len(df_containretra_copy)

# 删除 RYear-Year 小于 0 的行
df_containretra_copy = df_containretra_copy[df_containretra_copy['RYear-Year'] >= 0]

# 删除后行数
remaining_count = len(df_containretra_copy)

# 打印统计信息
print(f"原来有 {original_count} 行")
print(f"删除 RYear-Year < 0 的行数：{original_count - remaining_count}")
print(f"剩余 {remaining_count} 行")

原来有 363465 行
删除 RYear-Year < 0 的行数：143452
剩余 220013 行


### （4）删除自己本身也被撤稿的论文

In [47]:
#(4)The paper itself was retracted
#还有本身自己就被撤稿的论文
#检查 df_containretra_copy的id列中的id是否有出现在那4w篇撤稿文章id（df_retrapapers）中
#如果出现了就需要删除这些行
# 统计原始行数
original_count = len(df_containretra_copy)

# 删除df_containretra_copy中id在撤稿论文id列表中的行
df_containretra_copy = df_containretra_copy[~df_containretra_copy['id'].isin(df_retrapapers['id'])]

# 统计删减后行数
filtered_count = len(df_containretra_copy)

# 输出统计信息
print(f"原来有 {original_count} 行")
print(f"删除本身被撤稿的论文后，剩余 {filtered_count} 行")
print(f"共删除了 {original_count - filtered_count} 行")

原来有 220013 行
删除本身被撤稿的论文后，剩余 217296 行
共删除了 2717 行


### 检查每列NaN情况

In [48]:
# 计算每列的 NaN 数量
nan_counts = df_containretra_copy.isna().sum()

# 输出每列 NaN 的数量
print("每列 NaN 的数量：")
print(nan_counts)

每列 NaN 的数量：
id               0
reference_ids    0
Year             0
Journal_id       0
RPYear           0
RYear            0
RJournal_id      0
RYear-Year       0
dtype: int64


In [49]:
df_containretra3 = df_containretra_copy

### 保存清理后的df_containretra3

In [ ]:
output_path = '/Data4/yutao_wen/25.9.17/df_containretra3.csv'
# 保存为 CSV 文件
df_containretra3.to_csv(output_path, index=False)
'''
数据结构：
               id	reference_ids	Year	Journal_id	    RPYear	RYear	RJournal_id	RYear-Year
1	pub.1114696738	pub.1026667060	2015.0	jour.1050605	2015.0	2016.0	jour.1084523	1.0
2	pub.1114902564	pub.1028915017	2012.0	jour.1038288	1992.0	2016.0	jour.1077219	4.0
3	pub.1107325835	pub.1003143127	2018.0	jour.1030157	2006.0	2018.0	jour.1030157	0.0
10	pub.1134554602	pub.1106943956	2021.0	jour.1096497	2018.0	2021.0	jour.1115743	0.0
16	pub.1044393419	pub.1033427872	1998.0	jour.1082971	1997.0	1998.0	jour.1019114	0.0
...	...	...	...	...	...	...	...	... 
'''


# 模块二：Match四维度相同的对照组

## STEP1——创建四维度字典

### 读取原始字典文件

In [4]:
#Matching Variables ———— 构建四个维度的字典用于match
#(1)Year of publication 
#(2)Journal
#(3)Number of authors
#(4)Number of references

#这四个层面的字典数据有两个原始数据的字典
#字典一：papers_year_dic
#字典二：papers_journal_numA_numR_dic

#字典一位于路径“/Data4/yutao_wen/2part_Dic/total/papers_year_dic.json”
#字典二位于路径“/Data4/yutao_wen/2part_Dic/total/papers_journal_numA_numR_dic.json”


# 定义字典的文件路径
dic1_path = "/Data4/yutao_wen/2part_Dic/total/papers_year_dic.json"
dic2_path = "/Data4/yutao_wen/2part_Dic/total/papers_journal_numA_numR_dic.json"

In [5]:
# 读取字典1字典2
with open(dic1_path, "r", encoding="utf-8") as f:
    papers_year_dic = json.load(f)
with open(dic2_path, "r", encoding="utf-8") as f:
    papers_journal_numA_numR_dic = json.load(f)

### 检查两字典NaN数量

In [6]:
papers_year_dic

{'pub.1016604843': 2014.0,
 'pub.1116377458': 2006.0,
 'pub.1104981304': 2018.0,
 'pub.1081444315': 1966.0,
 'pub.1081455025': 1966.0,
 'pub.1149673930': 2019.0,
 'pub.1080136881': 1971.0,
 'pub.1089910710': 2004.0,
 'pub.1038660625': 2014.0,
 'pub.1031546016': 2014.0,
 'pub.1099400486': 2011.0,
 'pub.1128477735': 2011.0,
 'pub.1157607493': 2023.0,
 'pub.1147734828': 2022.0,
 'pub.1071656986': 2006.0,
 'pub.1127036348': 2019.0,
 'pub.1079912615': 1986.0,
 'pub.1079870344': 1986.0,
 'pub.1079814219': 1986.0,
 'pub.1079869103': 1986.0,
 'pub.1165742706': 2013.0,
 'pub.1125217894': 2008.0,
 'pub.1090249665': 2012.0,
 'pub.1103460844': 2012.0,
 'pub.1164701891': 2012.0,
 'pub.1075464740': 1953.0,
 'pub.1106873613': 2013.0,
 'pub.1101443626': 2013.0,
 'pub.1088522032': 2013.0,
 'pub.1117320944': 2018.0,
 'pub.1087354902': 2012.0,
 'pub.1046148457': 2012.0,
 'pub.1080957968': 1971.0,
 'pub.1080955749': 1971.0,
 'pub.1166093180': 2018.0,
 'pub.1154984190': 2018.0,
 'pub.1115092042': 1993.0,
 

In [7]:
# 统计 NaN 的数量
nan_count = sum(1 for year in papers_year_dic.values() if isinstance(year, float) and math.isnan(year))

print(f"papers_year_dic 中 NaN 的数量是: {nan_count}")

papers_year_dic 中 NaN 的数量是: 65868


In [8]:
papers_journal_numA_numR_dic

{'pub.1016604843': {'journal.id': nan, 'num_authors': 1, 'num_references': 0},
 'pub.1116377458': {'journal.id': 'jour.1368412',
  'num_authors': 1,
  'num_references': 0},
 'pub.1104981304': {'journal.id': nan, 'num_authors': 3, 'num_references': 0},
 'pub.1081444315': {'journal.id': 'jour.1398054',
  'num_authors': 4,
  'num_references': 0},
 'pub.1081455025': {'journal.id': 'jour.1399687',
  'num_authors': 1,
  'num_references': 0},
 'pub.1149673930': {'journal.id': nan, 'num_authors': 11, 'num_references': 0},
 'pub.1080136881': {'journal.id': 'jour.1081660',
  'num_authors': 4,
  'num_references': 0},
 'pub.1089910710': {'journal.id': nan, 'num_authors': 0, 'num_references': 0},
 'pub.1038660625': {'journal.id': nan,
  'num_authors': 0,
  'num_references': 253},
 'pub.1031546016': {'journal.id': nan, 'num_authors': 0, 'num_references': 2},
 'pub.1099400486': {'journal.id': nan, 'num_authors': 0, 'num_references': 0},
 'pub.1128477735': {'journal.id': nan, 'num_authors': 1, 'num_re

In [9]:
# 初始化 NaN 计数
nan_counts = {
    'journal.id': 0,
    'num_authors': 0,
    'num_references': 0
}

# 遍历 papers_journal_numA_numR_dic 进行 NaN 检查
for paper_id, info in papers_journal_numA_numR_dic.items():
    # 检查 journal.id 是否为 NaN
    if isinstance(info['journal.id'], float) and math.isnan(info['journal.id']):
        nan_counts['journal.id'] += 1
    
    # 检查 num_authors 是否为 NaN
    if isinstance(info['num_authors'], float) and math.isnan(info['num_authors']):
        nan_counts['num_authors'] += 1
    
    # 检查 num_references 是否为 NaN
    if isinstance(info['num_references'], float) and math.isnan(info['num_references']):
        nan_counts['num_references'] += 1

# 输出每个维度的 NaN 数量
print(f"journal.id 中 NaN 的数量: {nan_counts['journal.id']}")
print(f"num_authors 中 NaN 的数量: {nan_counts['num_authors']}")
print(f"num_references 中 NaN 的数量: {nan_counts['num_references']}")

journal.id 中 NaN 的数量: 24919078
num_authors 中 NaN 的数量: 0
num_references 中 NaN 的数量: 0


### 合并成四维度字典——9MIN

In [10]:
# 整合数据
merged_dic = {}

for paper_id in papers_year_dic:  # 以 papers_year_dic 为基准
    year_value = papers_year_dic.get(paper_id, 0)  # 先获取 year，可能是 NaN
    if isinstance(year_value, float) and math.isnan(year_value):  # 检测 NaN
        year_value = 0  # 将 NaN 转换为 0

    merged_dic[paper_id] = {
        "year": int(year_value),  # 确保 year 是整数
        "journal": papers_journal_numA_numR_dic.get(paper_id, {}).get('journal.id', 'Unknown'),
        "NumA": papers_journal_numA_numR_dic.get(paper_id, {}).get('num_authors', 0),
        "NumR": papers_journal_numA_numR_dic.get(paper_id, {}).get('num_references', 0),
    }

In [11]:
#命名为Contr_total_dic（作为匹配对照组的样本池）
Contr_total_dic = merged_dic

### 确认合并字典的NaN值与0值数量——4MIN

In [12]:
# 需要统计的键
keys_to_check = ['year', 'journal', 'NumA', 'NumR']

# 初始化计数器
nan_counts = Counter({key: 0 for key in keys_to_check})
zero_counts = Counter({key: 0 for key in keys_to_check})
#Contr_total_dic 是要遍历的字典
# 统计 NaN 和 0 值
for entry in Contr_total_dic.values():
    for key in keys_to_check:
        if key in entry:
            value = entry[key]
            
            # 检查 NaN 值
            if value is None or (isinstance(value, float) and math.isnan(value)):
                nan_counts[key] += 1
            
            # 检查 0 值（排除 NaN）
            elif value == 0:
                zero_counts[key] += 1

# 输出统计结果
for key in keys_to_check:
    print(f"{key} 中 NaN 的数量: {nan_counts[key]}")
    print(f"{key} 中 0 的数量: {zero_counts[key]}")

year 中 NaN 的数量: 0
year 中 0 的数量: 65868
journal 中 NaN 的数量: 24919078
journal 中 0 的数量: 0
NumA 中 NaN 的数量: 0
NumA 中 0 的数量: 12577964
NumR 中 NaN 的数量: 0
NumR 中 0 的数量: 70327440


### 排除数据残缺的部分（NaN与0）——4MIN

In [13]:
# 过滤掉包含 NaN 或 0 值的条目
Contr_total_dic = {
    k: v for k, v in Contr_total_dic.items()
    if all(
        v.get(key) is not None and
        not (isinstance(v[key], float) and math.isnan(v[key])) and  # 不为 NaN
        v[key] != 0  # 不为 0
        for key in ['year', 'journal', 'NumA', 'NumR']  # 只检查这四个维度
    )
}

### 剩余数据量展示

In [ ]:
#最后剩余的完整的数据数量
len(Contr_total_dic)

60326267

### 改进字典格式优化运算——14MIN

In [16]:
#现在我的字典形如：
"""
{"pub.1148484199": 
            {"year": 2020, 
             "journal": "jour.1047907",
             "NumA": 5, 
             "NumR": 50,},
 "pub.1148484455": 
            {"year": 2020, 
             "journal": "jour.1047907",
             "NumA": 5, 
             "NumR": 50,},...
 "pub.1148485584": 
            {"year": 2021, 
             "journal": "jour.1047955",
             "NumA": 3, 
             "NumR": 21,}
 "pub.1148485577": 
            {"year": 2021, 
             "journal": "jour.1047955",
             "NumA": 3, 
             "NumR": 21,}
                                        } """
#我希望将键值对进行互换，将year，journal，NumA，NumR相同的论文整合在一起，变成新的字典，
#即变成为：
'''
{{"year": 2020, "journal": "jour.1047907","NumA": 5, "NumR": 50,}:{pub.1148484199,pub.1148484455},
{"year": 2021, "journal": "jour.1047955","NumA": 3, "NumR": 21,}:{1148485584:1148485577}...}
'''

'\n{{"year": 2020, "journal": "jour.1047907","NumA": 5, "NumR": 50,}:{pub.1148484199,pub.1148484455},\n{"year": 2021, "journal": "jour.1047955","NumA": 3, "NumR": 21,}:{1148485584:1148485577}...}\n'

In [17]:
# 创建 defaultdict 来存储分组数据
grouped_data = defaultdict(set)

for pub_id, attributes in Contr_total_dic.items():
    key = tuple(sorted(attributes.items()))  # 关键修改：使用元组作为 key
    grouped_data[key].add(pub_id)

In [18]:
#数据展示
grouped_data

defaultdict(set,
            {(('NumA', 2),
              ('NumR', 2),
              ('journal', 'jour.1399801'),
              ('year', 2006)): {'pub.1071656983', 'pub.1071656986'},
             (('NumA', 2),
              ('NumR', 8),
              ('journal', 'jour.1453176'),
              ('year', 2013)): {'pub.1165742706'},
             (('NumA', 8),
              ('NumR', 2),
              ('journal', 'jour.1368781'),
              ('year', 2013)): {'pub.1114843554', 'pub.1114891884'},
             (('NumA', 6),
              ('NumR', 1),
              ('journal', 'jour.1142333'),
              ('year', 2013)): {'pub.1074030152'},
             (('NumA', 1),
              ('NumR', 3),
              ('journal', 'jour.1409145'),
              ('year', 2022)): {'pub.1153894014',
              'pub.1153894017',
              'pub.1153894020',
              'pub.1153894024'},
             (('NumA', 1),
              ('NumR', 1),
              ('journal', 'jour.1366435'),
              

## STEP2——匹配

### 获取目标论文ID列

In [ ]:
#现在总体的字典已经整理出来了，需要获取目标论文的id序列
#目标论文序列来源于文件：/Data4/yutao_wen/25.9.17/df_containretra3.csv
# 读取 CSV 文件
file_path = '/Data4/yutao_wen/25.9.17/df_containretra3.csv'
df = pd.read_csv(file_path)
# 提取 'id' 列并转换为列表
id_sequence = df['id'].tolist()


In [25]:
len(id_sequence)

217296

### 4维度Exactly匹配对照组

In [22]:
# 假设 id_sequence 是一个包含实验组 ID 的列表
id_sequence_set = set(id_sequence)  # 将 id_sequence 转换为集合

# 存储匹配结果
matched_data = []

# 遍历 grouped_data
for attributes, control_ids in grouped_data.items():
    # 还原元组为字典
    attributes_dict = dict(attributes)

    # 查找实验组 ID 是否存在
    common_ids = id_sequence_set.intersection(control_ids)
    if common_ids:
        for exp_id in common_ids:
            for control_id in control_ids:
                if exp_id != control_id:  # 过滤掉相同 ID
                    matched_data.append([
                        exp_id, attributes_dict["year"], attributes_dict["journal"], attributes_dict["NumA"], attributes_dict["NumR"],
                        control_id, attributes_dict["year"], attributes_dict["journal"], attributes_dict["NumA"], attributes_dict["NumR"]
                    ])

### 转换为Dataframe格式

In [23]:
# 转换为 DataFrame
df = pd.DataFrame(matched_data, columns=[
    "实验组ID", "year", "journal", "NumA", "NumR",
    "对照组ID", "year", "journal", "NumA", "NumR"
])

In [24]:
df

,实验组ID,year,journal,NumA,NumR,对照组ID,year,journal,NumA,NumR
0,pub.1114696738,2015,jour.1050605,1,7,pub.1114588196,2015,jour.1050605,1,7
1,pub.1002883431,2011,jour.1136657,1,2,pub.1008326753,2011,jour.1136657,1,2
2,pub.1002883431,2011,jour.1136657,1,2,pub.1009940677,2011,jour.1136657,1,2
3,pub.1002883431,2011,jour.1136657,1,2,pub.1026617007,2011,jour.1136657,1,2
4,pub.1002883431,2011,jour.1136657,1,2,pub.1042881218,2011,jour.1136657,1,2
...,...,...,...,...,...,...,...,...,...,...
1185726,pub.1041669049,2012,jour.1319913,5,40,pub.1010777462,2012,jour.1319913,5,40
1185727,pub.1103407823,2018,jour.1016375,3,57,pub.1106101540,2018,jour.1016375,3,57
1185728,pub.1028114604,2004,jour.1017523,3,38,pub.1032639142,2004,jour.1017523,3,38
1185729,pub.1041820780,2010,jour.1044317,3,13,pub.1041983066,2010,jour.1044317,3,13


In [22]:
#名称统一化
# 重命名
df = df.rename(columns={
    '实验组ID': 'id',
    'year': 'PYear',
    'journal': 'Journal_id',
    '对照组ID': 'Cid'
})

# 删除不需要的列
df = df.drop(columns=['year.1', 'journal.1', 'NumA.1', 'NumR.1'])

### 匹配文件保存

In [23]:
# 指定保存路径
save_path = "/Data4/yutao_wen/25.9.17/df_match.csv"

# 保存为 CSV 文件（不包含索引）
df.to_csv(save_path, index=False, encoding="utf-8")

# 模块三：Match事前事后citation

## STEP1——完善ECR三合一df文件

### 读取模块二匹配文件

In [24]:
df = pd.read_csv("/Data4/yutao_wen/25.9.17/df_match.csv")

In [25]:
df

,id,PYear,Journal_id,NumA,NumR,Cid
0,pub.1114696738,2015,jour.1050605,1,7,pub.1114588196
1,pub.1002883431,2011,jour.1136657,1,2,pub.1008326753
2,pub.1002883431,2011,jour.1136657,1,2,pub.1009940677
3,pub.1002883431,2011,jour.1136657,1,2,pub.1026617007
4,pub.1002883431,2011,jour.1136657,1,2,pub.1042881218
...,...,...,...,...,...,...
1185726,pub.1041669049,2012,jour.1319913,5,40,pub.1010777462
1185727,pub.1103407823,2018,jour.1016375,3,57,pub.1106101540
1185728,pub.1028114604,2004,jour.1017523,3,38,pub.1032639142
1185729,pub.1041820780,2010,jour.1044317,3,13,pub.1041983066


### 匹配文件描述性统计检查

In [26]:
# 设置 pandas 显示选项
pd.set_option('display.float_format', '{:.2f}'.format)
# 查看 describe()
df.describe()
#结果均显示正常

,PYear,NumA,NumR
count,1185731.00,1185731.00,1185731.00
mean,2015.00,4.06,22.46
std,6.55,2.59,25.50
min,1960.00,1.00,1.00
25%,2012.00,2.00,1.00
50%,2016.00,4.00,18.00
75%,2020.00,6.00,39.00
max,2022.00,58.00,809.00


### 完善匹配对信息（添加撤稿论文信息）

In [62]:
#需要一些撤稿论文的原始信息
#这些信息来自于模块二STEP3：/Data4/yutao_wen/25.9.17/df_containretra3.csv
df_containretra3 = pd.read_csv("/Data4/yutao_wen/25.9.17/df_containretra3.csv")

In [ ]:
#撤稿论文信息dataframe文件
df_containretra3

,id,reference_ids,Year,Journal_id,RPYear,RYear,RJournal_id,RYear-Year
0,pub.1114696738,pub.1026667060,2015.00,jour.1050605,2015.00,2016.00,jour.1084523,1.00
1,pub.1114902564,pub.1028915017,2012.00,jour.1038288,1992.00,2016.00,jour.1077219,4.00
2,pub.1107325835,pub.1003143127,2018.00,jour.1030157,2006.00,2018.00,jour.1030157,0.00
3,pub.1134554602,pub.1106943956,2021.00,jour.1096497,2018.00,2021.00,jour.1115743,0.00
4,pub.1044393419,pub.1033427872,1998.00,jour.1082971,1997.00,1998.00,jour.1019114,0.00
...,...,...,...,...,...,...,...,...
217291,pub.1092297051,pub.1015000895,2018.00,jour.1031133,2016.00,2019.00,jour.1116728,1.00
217292,pub.1013229134,pub.1003326122,2011.00,jour.1047723,2009.00,2016.00,jour.1121878,5.00
217293,pub.1016972058,pub.1008362454,2016.00,jour.1046517,2013.00,2018.00,jour.1327387,2.00
217294,pub.1085464507,pub.1039593849,2017.00,jour.1454776,2011.00,2018.00,jour.1082971,1.00


In [64]:
#实验组对照组配对信息dataframe文件
df

,id,PYear,Journal_id,NumA,NumR,Cid
0,pub.1114696738,2015,jour.1050605,1,7,pub.1114588196
1,pub.1002883431,2011,jour.1136657,1,2,pub.1008326753
2,pub.1002883431,2011,jour.1136657,1,2,pub.1009940677
3,pub.1002883431,2011,jour.1136657,1,2,pub.1026617007
4,pub.1002883431,2011,jour.1136657,1,2,pub.1042881218
...,...,...,...,...,...,...
1185726,pub.1041669049,2012,jour.1319913,5,40,pub.1010777462
1185727,pub.1103407823,2018,jour.1016375,3,57,pub.1106101540
1185728,pub.1028114604,2004,jour.1017523,3,38,pub.1032639142
1185729,pub.1041820780,2010,jour.1044317,3,13,pub.1041983066


In [65]:
#把 df_containretra3里的 reference_ids, RPYear, RYear, RJournal_id 按照id合并到df里
# 选择需要合并的几列
cols_to_merge = ['id', 'reference_ids', 'RPYear', 'RYear', 'RJournal_id']

# 执行合并
df_merged = df.merge(
    df_containretra3[cols_to_merge],
    on='id',
    how='left'   # 保留 df 的所有行
)


In [ ]:
#现在这个文件包含了
# （1）目标论文与控制组的一一对应关系
# （2）目标论文参考文献中被撤稿论文的：发表年份RPYear、撤稿年份RYear、来源期刊RJournal_id
df_merged

,id,PYear,Journal_id,NumA,NumR,Cid,reference_ids,RPYear,RYear,RJournal_id
0,pub.1114696738,2015,jour.1050605,1,7,pub.1114588196,pub.1026667060,2015.00,2016.00,jour.1084523
1,pub.1002883431,2011,jour.1136657,1,2,pub.1008326753,pub.1000478466,2011.00,2018.00,jour.1018957
2,pub.1002883431,2011,jour.1136657,1,2,pub.1009940677,pub.1000478466,2011.00,2018.00,jour.1018957
3,pub.1002883431,2011,jour.1136657,1,2,pub.1026617007,pub.1000478466,2011.00,2018.00,jour.1018957
4,pub.1002883431,2011,jour.1136657,1,2,pub.1042881218,pub.1000478466,2011.00,2018.00,jour.1018957
...,...,...,...,...,...,...,...,...,...,...
1185726,pub.1041669049,2012,jour.1319913,5,40,pub.1010777462,pub.1053707765,2010.00,2015.00,jour.1017016
1185727,pub.1103407823,2018,jour.1016375,3,57,pub.1106101540,pub.1006346538,2012.00,2019.00,jour.1042966
1185728,pub.1028114604,2004,jour.1017523,3,38,pub.1032639142,pub.1053211457,2001.00,2019.00,jour.1077138
1185729,pub.1041820780,2010,jour.1044317,3,13,pub.1041983066,pub.1023364708,2008.00,2012.00,jour.1138886


### 界定Now、T_PtoR、T_RtoN


In [ ]:
# 添加新列
df_merged = df_merged.assign(
    Now=2023,#为研究的截至点
    T_PtoR=df_merged['RYear'] - df_merged['PYear'],
    #目标论文发表到参考文献撤稿时间区间T
    T_RtoN=2023 - df_merged['RYear']
    #参考文献撤稿到2023时间区间T
)


In [69]:
#命名为df_merged_ECR
#Experimental Group---Control Group---Retratepapers
df_merged_ECR = df_merged

In [70]:
df_merged_ECR

,id,PYear,Journal_id,NumA,NumR,Cid,reference_ids,RPYear,RYear,RJournal_id,Now,T_PtoR,T_RtoN
0,pub.1114696738,2015,jour.1050605,1,7,pub.1114588196,pub.1026667060,2015.00,2016.00,jour.1084523,2023,1.00,7.00
1,pub.1002883431,2011,jour.1136657,1,2,pub.1008326753,pub.1000478466,2011.00,2018.00,jour.1018957,2023,7.00,5.00
2,pub.1002883431,2011,jour.1136657,1,2,pub.1009940677,pub.1000478466,2011.00,2018.00,jour.1018957,2023,7.00,5.00
3,pub.1002883431,2011,jour.1136657,1,2,pub.1026617007,pub.1000478466,2011.00,2018.00,jour.1018957,2023,7.00,5.00
4,pub.1002883431,2011,jour.1136657,1,2,pub.1042881218,pub.1000478466,2011.00,2018.00,jour.1018957,2023,7.00,5.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1185726,pub.1041669049,2012,jour.1319913,5,40,pub.1010777462,pub.1053707765,2010.00,2015.00,jour.1017016,2023,3.00,8.00
1185727,pub.1103407823,2018,jour.1016375,3,57,pub.1106101540,pub.1006346538,2012.00,2019.00,jour.1042966,2023,1.00,4.00
1185728,pub.1028114604,2004,jour.1017523,3,38,pub.1032639142,pub.1053211457,2001.00,2019.00,jour.1077138,2023,15.00,4.00
1185729,pub.1041820780,2010,jour.1044317,3,13,pub.1041983066,pub.1023364708,2008.00,2012.00,jour.1138886,2023,2.00,11.00


### 添加实验组控制组对应编号

In [71]:
# 1. 生成实验组编号 E000001, E000002, ...
df_merged_ECR['orderE'] = 'E' + (df_merged_ECR['id'].astype('category').cat.codes + 1).astype(str).str.zfill(6)

# 2. 创建对照组编号 E000001_1, E000001_2, ...
df_merged_ECR['orderC'] = df_merged_ECR.groupby('id').cumcount() + 1
df_merged_ECR['orderC'] = df_merged_ECR['orderE'] + '_' + df_merged_ECR['orderC'].astype(str)


In [ ]:
#将orderE放在id前，orderC放在Cid前方便观察
# 指定新顺序
new_order = [
    'orderE', 'id', 'PYear', 'Journal_id', 'NumA', 'NumR',
    'orderC', 'Cid', 'reference_ids', 'RPYear', 'RYear',
    'RJournal_id', 'Now', 'T_PtoR', 'T_RtoN'
]
# 重新排列列顺序
df_merged_ECR = df_merged_ECR[new_order]

# 按照 orderE 升序排序，并重置索引
df_merged_ECR = df_merged_ECR.sort_values(by='orderE').reset_index(drop=True)


In [74]:
df_merged_ECR

,orderE,id,PYear,Journal_id,NumA,NumR,orderC,Cid,reference_ids,RPYear,RYear,RJournal_id,Now,T_PtoR,T_RtoN
0,E000001,pub.1000002908,2008,jour.1092102,7,50,E000001_2,pub.1037416417,pub.1007764568,2002.00,2015.00,jour.1017574,2023,7.00,8.00
1,E000001,pub.1000002908,2008,jour.1092102,7,50,E000001_1,pub.1022133001,pub.1007764568,2002.00,2015.00,jour.1017574,2023,7.00,8.00
2,E000002,pub.1000005181,2013,jour.1077138,2,54,E000002_1,pub.1027543414,pub.1004718702,2007.00,2017.00,jour.1099159,2023,4.00,6.00
3,E000002,pub.1000005181,2013,jour.1077138,2,54,E000002_2,pub.1022003078,pub.1004718702,2007.00,2017.00,jour.1099159,2023,4.00,6.00
4,E000002,pub.1000005181,2013,jour.1077138,2,54,E000002_3,pub.1021379269,pub.1004718702,2007.00,2017.00,jour.1099159,2023,4.00,6.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1185726,E095414,pub.1164912181,1995,jour.1023000,2,3,E095414_1,pub.1070561906,pub.1033569250,1991.00,1996.00,jour.1312259,2023,1.00,27.00
1185727,E095415,pub.1164948030,2020,jour.1320609,11,37,E095415_1,pub.1129351535,pub.1126668311,2020.00,2020.00,jour.1297452,2023,0.00,3.00
1185728,E095416,pub.1166136474,2001,jour.1086802,4,42,E095416_2,pub.1032595242,pub.1070819411,1995.00,2020.00,jour.1017902,2023,19.00,3.00
1185729,E095416,pub.1166136474,2001,jour.1086802,4,42,E095416_1,pub.1047614739,pub.1070819411,1995.00,2020.00,jour.1017902,2023,19.00,3.00


### 保存备份文件


In [75]:
output_path = '/Data4/yutao_wen/25.9.17/df_merged_ECR.csv'
# 保存为 CSV 文件
df_merged_ECR.to_csv(output_path, index=False)

In [2]:
df_merged_ECR = pd.read_csv("/Data4/yutao_wen/25.9.17/df_merged_ECR.csv")

## STEP2——事前YearlyCitation计算

### 拆分实验组对照组

In [3]:
#选择实验组的列
df_exper = df_merged_ECR[["orderE","id", "PYear", "Journal_id", "NumA", "NumR","RYear","Now","T_PtoR","T_RtoN"]].copy()
df_contr = df_merged_ECR[["orderC","Cid", "PYear", "Journal_id", "NumA", "NumR","RYear","Now","T_PtoR","T_RtoN"]].copy()

In [4]:
# 去除 df_exper 中的重复行减小后续运算量
df_exper = df_exper.drop_duplicates()

In [5]:
df_exper

,orderE,id,PYear,Journal_id,NumA,NumR,RYear,Now,T_PtoR,T_RtoN
0,E000001,pub.1000002908,2008,jour.1092102,7,50,2015.0,2023,7.0,8.0
2,E000002,pub.1000005181,2013,jour.1077138,2,54,2017.0,2023,4.0,6.0
8,E000003,pub.1000005249,2012,jour.1037553,10,90,2012.0,2023,0.0,11.0
9,E000004,pub.1000005788,2002,jour.1014075,2,101,2011.0,2023,9.0,12.0
10,E000005,pub.1000005995,2013,jour.1135213,5,34,2022.0,2023,9.0,1.0
...,...,...,...,...,...,...,...,...,...,...
1185724,E095413,pub.1164807166,2005,jour.1096580,8,40,2016.0,2023,11.0,7.0
1185726,E095414,pub.1164912181,1995,jour.1023000,2,3,1996.0,2023,1.0,27.0
1185727,E095415,pub.1164948030,2020,jour.1320609,11,37,2020.0,2023,0.0,3.0
1185728,E095416,pub.1166136474,2001,jour.1086802,4,42,2020.0,2023,19.0,3.0


In [6]:
df_contr

,orderC,Cid,PYear,Journal_id,NumA,NumR,RYear,Now,T_PtoR,T_RtoN
0,E000001_2,pub.1037416417,2008,jour.1092102,7,50,2015.0,2023,7.0,8.0
1,E000001_1,pub.1022133001,2008,jour.1092102,7,50,2015.0,2023,7.0,8.0
2,E000002_1,pub.1027543414,2013,jour.1077138,2,54,2017.0,2023,4.0,6.0
3,E000002_2,pub.1022003078,2013,jour.1077138,2,54,2017.0,2023,4.0,6.0
4,E000002_3,pub.1021379269,2013,jour.1077138,2,54,2017.0,2023,4.0,6.0
...,...,...,...,...,...,...,...,...,...,...
1185726,E095414_1,pub.1070561906,1995,jour.1023000,2,3,1996.0,2023,1.0,27.0
1185727,E095415_1,pub.1129351535,2020,jour.1320609,11,37,2020.0,2023,0.0,3.0
1185728,E095416_2,pub.1032595242,2001,jour.1086802,4,42,2020.0,2023,19.0,3.0
1185729,E095416_1,pub.1047614739,2001,jour.1086802,4,42,2020.0,2023,19.0,3.0


### 拆分为Tdiff形式(撤稿前)


In [7]:
# 实验组E撤稿前的expanded_df
# 新建一个空列表，用来存放拆分后的数据行
expanded_rows_E = []

# 遍历原始 DataFrame 的每一行
for _, row in df_exper.iterrows():
    # 获取起始年份 PYear 和 终止年份 RYear（转换为 int）
    start_year = int(row['PYear'])
    end_year = int(row['RYear'])
    
    # 遍历区间，每年生成一行
    for y in range(start_year, end_year):
        # 生成 intervalB 列的值，如 [2006,2007]
        intervalB = f"[{y},{y + 1}]"
        # 计算 Tdiff 列的值（区间上界 - RYear）
        Tdiff = y - end_year
        
        # 创建新行，复制原有行数据，并插入 intervalB 和 Tdiff
        new_row = row.copy()
        new_row['intervalB'] = intervalB
        new_row['Tdiff'] = Tdiff
        
        # 将新行加入 expanded_rows 列表
        expanded_rows_E.append(new_row)

# 将拆分后的数据转换为新的 DataFrame
expanded_df_EB = pd.DataFrame(expanded_rows_E)

# 重置索引，保持整洁
expanded_df_EB.reset_index(drop=True, inplace=True)


In [11]:
# 控制组c撤稿前的expanded_df
# 新建一个空列表，用来存放拆分后的数据行
expanded_rows_C = []

# 遍历原始 DataFrame 的每一行
for _, row in df_contr.iterrows():
    # 获取起始年份 PYear 和 终止年份 RYear（转换为 int）
    start_year = int(row['PYear'])
    end_year = int(row['RYear'])
    
    # 遍历区间，每年生成一行
    for y in range(start_year, end_year):
        # 生成 intervalB 列的值，如 [2006,2007]
        intervalB = f"[{y},{y + 1}]"
        # 计算 Tdiff 列的值（区间上界 - RYear）
        Tdiff = y - end_year
        
        # 创建新行，复制原有行数据，并插入 intervalB 和 Tdiff
        new_row = row.copy()
        new_row['intervalB'] = intervalB
        new_row['Tdiff'] = Tdiff
        
        # 将新行加入 expanded_rows 列表
        expanded_rows_C.append(new_row)

# 将拆分后的数据转换为新的 DataFrame
expanded_df_CB = pd.DataFrame(expanded_rows_C)

# 重置索引，保持整洁
expanded_df_CB.reset_index(drop=True, inplace=True)


### 读取全年份引用关系字典

In [8]:
dic2_path = "/Data4/yutao_wen/ALLyear_paper_refs.json"
# 读取字典2
with open(dic2_path, "r", encoding="utf-8") as f:
    ALLyear_paper_refs = json.load(f)

### yearly字典序列创建

In [9]:
# 实验组E：
# 创建一个空字典来存放键值对
interval_dict_E = {}

# 遍历实验组expanded_df_EB 的每一行，提取id、Tdiff 和 intervalB 列
for _, row in expanded_df_EB.iterrows():
    # 创建键：id + '_' + Tdiff
    key = f"{row['id']}_{row['Tdiff']}"
    # 将 intervalB 作为值存入字典
    interval_dict_E[key] = row['intervalB']


In [12]:
# 控制组C：
# 创建一个空字典来存放键值对
interval_dict_C = {}

# 遍历控制组expanded_df_CB 的每一行，提取Cid、Tdiff 和 intervalB 列
for _, row in expanded_df_CB.iterrows():
    # 创建键：id + '_' + Tdiff
    key = f"{row['Cid']}_{row['Tdiff']}"
    # 将 intervalB 作为值存入字典
    interval_dict_C[key] = row['intervalB']

### 实验组撤稿前yearlycitation匹配+保存

In [10]:
# 实验组E:
# 构造：纯ID -> [(完整键, (lower, upper)), ...]
paper_intervals = {}
for full_key, value in interval_dict_E.items():
    # full_key 形如 'pub.1000000694_-4'
    # 提取纯ID：先按 "_" 分割，再去除前缀 "pub."
    pure_id = full_key.split('_')[0][4:]
    
    # 解析 value 字符串，如 '[2006,2007]' 去掉中括号后按逗号分割
    lower_str, upper_str = value.strip('[]').split(',')
    lower = float(lower_str)
    upper = float(upper_str)
    
    # 将数据存入字典
    if pure_id not in paper_intervals:
        paper_intervals[pure_id] = []
    paper_intervals[pure_id].append((full_key, (lower, upper)))

# 初始化统计字典，key 为 interval_dict_E 中的完整键，值初始化为 0
Ecitation_count_BeforeRe = {key: 0 for key in interval_dict_E.keys()}

# 遍历 ALLyear_paper_refs 进行引用统计
for citing_paper, refs in tqdm(ALLyear_paper_refs.items(), desc="Processing citations"):
    try:
        # 从引用论文ID中提取年份（格式如 "1013225891_1900"）
        _, year_str = citing_paper.split('_', 1)
        year = float(year_str)
    except Exception:
        continue  # 跳过格式错误的数据

    # 对每个参考文献进行处理
    for ref in refs:
        # 提取被引用论文的纯ID。假设格式为 "1005974208" 或 "1005974208_其他信息"
        ref_id = ref.partition('_')[0]
        
        if ref_id in paper_intervals:
            # 检查该论文所有的区间是否满足引用年份条件
            for full_key, (lower, upper) in paper_intervals[ref_id]:
                if lower <= year <= upper:
                    Ecitation_count_BeforeRe[full_key] += 1

Processing citations: 100%|██████████| 71184251/71184251 [10:42<00:00, 110870.36it/s]


In [15]:
#保存结果
output_path = '/Data4/yutao_wen/25.9.17/Ecitation_count_BeforeRe.json'
# 保存字典
with open(output_path, 'w') as f:
    json.dump(Ecitation_count_BeforeRe, f, indent=4)

### 控制组撤稿前yearlycitation匹配+保存

In [13]:
# 实验组C:
# 构造：纯ID -> [(完整键, (lower, upper)), ...]
paper_intervals = {}
for full_key, value in interval_dict_C.items():
    # full_key 形如 'pub.1000000694_-4'
    # 提取纯ID：先按 "_" 分割，再去除前缀 "pub."
    pure_id = full_key.split('_')[0][4:]
    
    # 解析 value 字符串，如 '[2006,2007]' 去掉中括号后按逗号分割
    lower_str, upper_str = value.strip('[]').split(',')
    lower = float(lower_str)
    upper = float(upper_str)
    
    # 将数据存入字典
    if pure_id not in paper_intervals:
        paper_intervals[pure_id] = []
    paper_intervals[pure_id].append((full_key, (lower, upper)))

# 初始化统计字典，key 为 interval_dict_C 中的完整键，值初始化为 0
Ccitation_count_BeforeRe = {key: 0 for key in interval_dict_C.keys()}

# 遍历 ALLyear_paper_refs 进行引用统计
for citing_paper, refs in tqdm(ALLyear_paper_refs.items(), desc="Processing citations"):
    try:
        # 从引用论文ID中提取年份（格式如 "1013225891_1900"）
        _, year_str = citing_paper.split('_', 1)
        year = float(year_str)
    except Exception:
        continue  # 跳过格式错误的数据

    # 对每个参考文献进行处理
    for ref in refs:
        # 提取被引用论文的纯ID。假设格式为 "1005974208" 或 "1005974208_其他信息"
        ref_id = ref.partition('_')[0]
        
        if ref_id in paper_intervals:
            # 检查该论文所有的区间是否满足引用年份条件
            for full_key, (lower, upper) in paper_intervals[ref_id]:
                if lower <= year <= upper:
                    Ccitation_count_BeforeRe[full_key] += 1

Processing citations: 100%|██████████| 71184251/71184251 [12:21<00:00, 96056.87it/s] 


In [16]:
#保存结果
output_path = '/Data4/yutao_wen/25.9.17/Ccitation_count_BeforeRe.json'
# 保存字典
with open(output_path, 'w') as f:
    json.dump(Ccitation_count_BeforeRe, f, indent=4)

In [18]:
#导入保存的数据
Ecitation_count_BeforeRe_path = "/Data4/yutao_wen/25.9.17/Ecitation_count_BeforeRe.json"
Ccitation_count_BeforeRe_path = "/Data4/yutao_wen/25.9.17/Ccitation_count_BeforeRe.json"
with open(Ecitation_count_BeforeRe_path, "r", encoding="utf-8") as f:
    Ecitation_count_BeforeRe = json.load(f)
with open(Ccitation_count_BeforeRe_path, "r", encoding="utf-8") as f:
    Ccitation_count_BeforeRe = json.load(f)

## STEP3——事后YearlyCitation计算

### 拆分为Tdiff形式(撤稿后)

In [22]:
# 实验组E撤稿后的expanded_df
# 新建一个空列表，用来存放拆分后的数据行
expanded_rows_E = []

# 遍历原始 DataFrame 的每一行
for _, row in df_exper.iterrows():
    # 获取起始年份 RYear 和 终止年份 Now（转换为 int）
    start_year = int(row['RYear'])
    end_year = int(row['Now'])
    
    # 遍历区间，每年生成一行
    for y in range(start_year, end_year):
        # 生成 intervalA 列的值，如 [2006,2007]
        intervalA = f"[{y},{y + 1}]"
        # 计算 Tdiff 列的值（区间上界 - RYear）
        Tdiff = y - end_year
        
        # 创建新行，复制原有行数据，并插入 intervalA 和 Tdiff
        new_row = row.copy()
        new_row['intervalA'] = intervalA
        new_row['Tdiff'] = Tdiff
        
        # 将新行加入 expanded_rows 列表
        expanded_rows_E.append(new_row)

# 将拆分后的数据转换为新的 DataFrame
expanded_df_EA = pd.DataFrame(expanded_rows_E)

# 重置索引，保持整洁
expanded_df_EA.reset_index(drop=True, inplace=True)


In [26]:
# 控制组c撤稿后的expanded_df
# 新建一个空列表，用来存放拆分后的数据行
expanded_rows_C = []

# 遍历原始 DataFrame 的每一行
for _, row in df_contr.iterrows():
    # 获取起始年份 RYear 和 终止年份 Now（转换为 int）
    start_year = int(row['RYear'])
    end_year = int(row['Now'])
    
    # 遍历区间，每年生成一行
    for y in range(start_year, end_year):
        # 生成 intervalA 列的值，如 [2006,2007]
        intervalA = f"[{y},{y + 1}]"
        # 计算 Tdiff 列的值（区间上界 - RYear）
        Tdiff = y - end_year
        
        # 创建新行，复制原有行数据，并插入 intervalA 和 Tdiff
        new_row = row.copy()
        new_row['intervalA'] = intervalA
        new_row['Tdiff'] = Tdiff
        
        # 将新行加入 expanded_rows 列表
        expanded_rows_C.append(new_row)

# 将拆分后的数据转换为新的 DataFrame
expanded_df_CA = pd.DataFrame(expanded_rows_C)

# 重置索引，保持整洁
expanded_df_CA.reset_index(drop=True, inplace=True)


### yearly字典序列创建

In [23]:
# 实验组E：
# 创建一个空字典来存放键值对
interval_dict_E = {}

# 遍历实验组expanded_df_EA 的每一行，提取id、Tdiff 和 intervalA 列
for _, row in expanded_df_EA.iterrows():
    # 创建键：id + '_' + Tdiff
    key = f"{row['id']}_{row['Tdiff']}"
    # 将 intervalA 作为值存入字典
    interval_dict_E[key] = row['intervalA']

In [27]:
# 控制组C：
# 创建一个空字典来存放键值对
interval_dict_C = {}

# 遍历控制组expanded_df_CA 的每一行，提取Cid、Tdiff 和 intervalA 列
for _, row in expanded_df_CA.iterrows():
    # 创建键：id + '_' + Tdiff
    key = f"{row['Cid']}_{row['Tdiff']}"
    # 将 intervalA 作为值存入字典
    interval_dict_C[key] = row['intervalA']

### 实验组撤稿后yearlycitation匹配+保存

In [24]:
# 实验组E:
# 构造：纯ID -> [(完整键, (lower, upper)), ...]
paper_intervals = {}
for full_key, value in interval_dict_E.items():
    # full_key 形如 'pub.1000000694_-4'
    # 提取纯ID：先按 "_" 分割，再去除前缀 "pub."
    pure_id = full_key.split('_')[0][4:]
    
    # 解析 value 字符串，如 '[2006,2007]' 去掉中括号后按逗号分割
    lower_str, upper_str = value.strip('[]').split(',')
    lower = float(lower_str)
    upper = float(upper_str)
    
    # 将数据存入字典
    if pure_id not in paper_intervals:
        paper_intervals[pure_id] = []
    paper_intervals[pure_id].append((full_key, (lower, upper)))

# 初始化统计字典，key 为 interval_dict_E 中的完整键，值初始化为 0
Ecitation_count_AfterRe = {key: 0 for key in interval_dict_E.keys()}

# 遍历 ALLyear_paper_refs 进行引用统计
for citing_paper, refs in tqdm(ALLyear_paper_refs.items(), desc="Processing citations"):
    try:
        # 从引用论文ID中提取年份（格式如 "1013225891_1900"）
        _, year_str = citing_paper.split('_', 1)
        year = float(year_str)
    except Exception:
        continue  # 跳过格式错误的数据

    # 对每个参考文献进行处理
    for ref in refs:
        # 提取被引用论文的纯ID。假设格式为 "1005974208" 或 "1005974208_其他信息"
        ref_id = ref.partition('_')[0]
        
        if ref_id in paper_intervals:
            # 检查该论文所有的区间是否满足引用年份条件
            for full_key, (lower, upper) in paper_intervals[ref_id]:
                if lower <= year <= upper:
                    Ecitation_count_AfterRe[full_key] += 1

Processing citations: 100%|██████████| 71184251/71184251 [09:56<00:00, 119290.31it/s]


In [25]:
#保存结果
output_path = '/Data4/yutao_wen/25.9.17/Ecitation_count_AfterRe.json'
# 保存字典
with open(output_path, 'w') as f:
    json.dump(Ecitation_count_AfterRe, f, indent=4)

### 控制组撤稿后yearlycitation匹配+保存

In [28]:
# 实验组C:
# 构造：纯ID -> [(完整键, (lower, upper)), ...]
paper_intervals = {}
for full_key, value in interval_dict_C.items():
    # full_key 形如 'pub.1000000694_-4'
    # 提取纯ID：先按 "_" 分割，再去除前缀 "pub."
    pure_id = full_key.split('_')[0][4:]
    
    # 解析 value 字符串，如 '[2006,2007]' 去掉中括号后按逗号分割
    lower_str, upper_str = value.strip('[]').split(',')
    lower = float(lower_str)
    upper = float(upper_str)
    
    # 将数据存入字典
    if pure_id not in paper_intervals:
        paper_intervals[pure_id] = []
    paper_intervals[pure_id].append((full_key, (lower, upper)))

# 初始化统计字典，key 为 interval_dict_C 中的完整键，值初始化为 0
Ccitation_count_AfterRe = {key: 0 for key in interval_dict_C.keys()}

# 遍历 ALLyear_paper_refs 进行引用统计
for citing_paper, refs in tqdm(ALLyear_paper_refs.items(), desc="Processing citations"):
    try:
        # 从引用论文ID中提取年份（格式如 "1013225891_1900"）
        _, year_str = citing_paper.split('_', 1)
        year = float(year_str)
    except Exception:
        continue  # 跳过格式错误的数据

    # 对每个参考文献进行处理
    for ref in refs:
        # 提取被引用论文的纯ID。假设格式为 "1005974208" 或 "1005974208_其他信息"
        ref_id = ref.partition('_')[0]
        
        if ref_id in paper_intervals:
            # 检查该论文所有的区间是否满足引用年份条件
            for full_key, (lower, upper) in paper_intervals[ref_id]:
                if lower <= year <= upper:
                    Ccitation_count_AfterRe[full_key] += 1

Processing citations: 100%|██████████| 71184251/71184251 [13:19<00:00, 88987.25it/s] 


In [29]:
#保存结果
output_path = '/Data4/yutao_wen/25.9.17/Ccitation_count_AfterRe.json'
# 保存字典
with open(output_path, 'w') as f:
    json.dump(Ccitation_count_AfterRe, f, indent=4)

## STEP4——xxx

### 导入STEP2\STEP3的yearlycitation数据

In [3]:
#实验组对照组事前yearlycitation
Ecitation_count_BeforeRe_path = "/Data4/yutao_wen/25.9.17/Ecitation_count_BeforeRe.json"
Ccitation_count_BeforeRe_path = "/Data4/yutao_wen/25.9.17/Ccitation_count_BeforeRe.json"
with open(Ecitation_count_BeforeRe_path, "r", encoding="utf-8") as f:
    Ecitation_count_BeforeRe = json.load(f)
with open(Ccitation_count_BeforeRe_path, "r", encoding="utf-8") as f:
    Ccitation_count_BeforeRe = json.load(f)

In [4]:
#实验组对照组事前yearlycitation
Ecitation_count_AfterRe_path = "/Data4/yutao_wen/25.9.17/Ecitation_count_AfterRe.json"
Ccitation_count_AfterRe_path = "/Data4/yutao_wen/25.9.17/Ccitation_count_AfterRe.json"
with open(Ecitation_count_AfterRe_path, "r", encoding="utf-8") as f:
    Ecitation_count_AfterRe = json.load(f)
with open(Ccitation_count_AfterRe_path, "r", encoding="utf-8") as f:
    Ccitation_count_AfterRe = json.load(f)

### yearlycitation数据整合为df结构

In [ ]:
# 实验组撤稿前
# 初始化实验组合并后的字典
merged_Ecitation_count_BeforeRe = defaultdict(list)

# 遍历原始字典，按pub分组
for key, value in Ecitation_count_BeforeRe.items():
    pub, year = key.rsplit('_', 1)  # 分割字符串，提取pub和年份
    merged_Ecitation_count_BeforeRe[pub].append((int(year), value))

# 按年份排序，并转换为元组形式
for pub in merged_Ecitation_count_BeforeRe:
    merged_Ecitation_count_BeforeRe[pub].sort(key=lambda x: x[0])  # 根据年份排序
    merged_Ecitation_count_BeforeRe[pub] = tuple(val for _, val in merged_Ecitation_count_BeforeRe[pub])

In [ ]:
# 控制组+撤稿前
# 初始化控制组合并后的字典
merged_Ccitation_count_BeforeRe = defaultdict(list)

# 遍历原始字典，按pub分组
for key, value in Ccitation_count_BeforeRe.items():
    pub, year = key.rsplit('_', 1)  # 分割字符串，提取pub和年份
    merged_Ccitation_count_BeforeRe[pub].append((int(year), value))

# 按年份排序，并转换为元组形式
for pub in merged_Ccitation_count_BeforeRe:
    merged_Ccitation_count_BeforeRe[pub].sort(key=lambda x: x[0])  # 根据年份排序
    merged_Ccitation_count_BeforeRe[pub] = tuple(val for _, val in merged_Ccitation_count_BeforeRe[pub])

In [ ]:
# 实验组+撤稿后
# 初始化实验组合并后的字典
merged_Ecitation_count_AfterRe = defaultdict(list)

# 遍历原始字典，按pub分组
for key, value in Ecitation_count_AfterRe.items():
    pub, year = key.rsplit('_', 1)  # 分割字符串，提取pub和年份
    merged_Ecitation_count_AfterRe[pub].append((int(year), value))

# 按年份排序，并转换为元组形式
for pub in merged_Ecitation_count_AfterRe:
    merged_Ecitation_count_AfterRe[pub].sort(key=lambda x: x[0])  # 根据年份排序
    merged_Ecitation_count_AfterRe[pub] = tuple(val for _, val in merged_Ecitation_count_AfterRe[pub])

In [ ]:
# 控制组+撤稿前
# 初始化控制组合并后的字典
merged_Ccitation_count_AfterRe = defaultdict(list)

# 遍历原始字典，按pub分组
for key, value in Ccitation_count_AfterRe.items():
    pub, year = key.rsplit('_', 1)  # 分割字符串，提取pub和年份
    merged_Ccitation_count_AfterRe[pub].append((int(year), value))

# 按年份排序，并转换为元组形式
for pub in merged_Ccitation_count_AfterRe:
    merged_Ccitation_count_AfterRe[pub].sort(key=lambda x: x[0])  # 根据年份排序
    merged_Ccitation_count_AfterRe[pub] = tuple(val for _, val in merged_Ccitation_count_AfterRe[pub])

### 合并4维度exactly匹配关系与yearlycitation

In [27]:
#导入四维度exactly相等文件
#模块二最终的四维度相同文件
df_match = pd.read_csv("/Data4/yutao_wen/25.9.17/df_match.csv")

In [28]:
df_match

,id,PYear,Journal_id,NumA,NumR,Cid
0,pub.1114696738,2015,jour.1050605,1,7,pub.1114588196
1,pub.1002883431,2011,jour.1136657,1,2,pub.1008326753
2,pub.1002883431,2011,jour.1136657,1,2,pub.1009940677
3,pub.1002883431,2011,jour.1136657,1,2,pub.1026617007
4,pub.1002883431,2011,jour.1136657,1,2,pub.1042881218
...,...,...,...,...,...,...
1185726,pub.1041669049,2012,jour.1319913,5,40,pub.1010777462
1185727,pub.1103407823,2018,jour.1016375,3,57,pub.1106101540
1185728,pub.1028114604,2004,jour.1017523,3,38,pub.1032639142
1185729,pub.1041820780,2010,jour.1044317,3,13,pub.1041983066


In [ ]:
#提取出对应关系列
E_C_df = df_match[["id", "Cid"]]

In [30]:
#添加撤稿前后的yearlycitation数据
E_C_df["Ecitation_Before"] = E_C_df["id"].map(merged_Ecitation_count_BeforeRe)
E_C_df["Ccitation_Before"] = E_C_df["Cid"].map(merged_Ccitation_count_BeforeRe)
E_C_df["Ecitation_After"] = E_C_df["id"].map(merged_Ecitation_count_AfterRe)
E_C_df["Ccitation_After"] = E_C_df["Cid"].map(merged_Ccitation_count_AfterRe)

/tmp/ipykernel_2001055/2034632421.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  E_C_df["Ecitation_Before"] = E_C_df["id"].map(merged_Ecitation_count_BeforeRe)
/tmp/ipykernel_2001055/2034632421.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  E_C_df["Ccitation_Before"] = E_C_df["Cid"].map(merged_Ccitation_count_BeforeRe)
/tmp/ipykernel_2001055/2034632421.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value i

In [ ]:
#该数据即整合了
E_C_df

,id,Cid,Ecitation_Before,Ccitation_Before,Ecitation_After,Ccitation_After
0,pub.1114696738,pub.1114588196,"(0,)","(0,)","(0, 0, 0, 0, 0, 0, 0)","(0, 0, 0, 0, 0, 0, 0)"
1,pub.1002883431,pub.1008326753,"(0, 0, 0, 0, 0, 0, 0)","(0, 0, 0, 0, 0, 0, 0)","(0, 0, 0, 0, 0)","(0, 0, 0, 0, 0)"
2,pub.1002883431,pub.1009940677,"(0, 0, 0, 0, 0, 0, 0)","(0, 0, 0, 0, 0, 0, 0)","(0, 0, 0, 0, 0)","(0, 0, 0, 0, 0)"
3,pub.1002883431,pub.1026617007,"(0, 0, 0, 0, 0, 0, 0)","(0, 0, 0, 0, 0, 1, 1)","(0, 0, 0, 0, 0)","(0, 0, 0, 0, 0)"
4,pub.1002883431,pub.1042881218,"(0, 0, 0, 0, 0, 0, 0)","(0, 0, 1, 1, 0, 0, 0)","(0, 0, 0, 0, 0)","(0, 0, 0, 0, 0)"
...,...,...,...,...,...,...
1185726,pub.1041669049,pub.1010777462,"(3, 10, 9)","(10, 18, 15)","(5, 4, 2, 2, 3, 3, 3, 3)","(10, 11, 9, 4, 2, 7, 7, 2)"
1185727,pub.1103407823,pub.1106101540,"(4,)","(2,)","(19, 33, 37, 37)","(5, 5, 4, 6)"
1185728,pub.1028114604,pub.1032639142,"(2, 2, 0, 1, 1, 0, 0, 0, 2, 2, 0, 0, 0, 0, 1)","(10, 7, 5, 8, 9, 5, 2, 7, 8, 10, 12, 6, 7, 14,...","(1, 0, 0, 0)","(5, 9, 6, 3)"
1185729,pub.1041820780,pub.1041983066,"(1, 0)","(0, 0)","(0, 1, 2, 2, 2, 3, 4, 3, 3, 4, 5)","(1, 2, 2, 2, 3, 2, 0, 2, 3, 3, 4)"
